## C S 329E HW 7

# Naive Bayes 

## Your name here (and your partner's name if you are working in a pair)
Yixing Ma
Daniel Lam
group 31

For this week's homework we are going explore one new classification technique:

  - Naive Bayes

We are reusing the version of the Melbourne housing data set from HW6, to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

In addition to building our own Naive Bayes classifier, we are going to compare the performace of our classifier to the [Gaussian Naive Bayes Classifier](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) available in the scikit-learn library. 

In [1]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
from sklearn.naive_bayes import GaussianNB # The only thing in scikit-learn you can use this assignment

# Starting off loading a training set and setting a variable for the target column, "Type"
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')
target_col = 'Type'

## Q1 - Fix a column of data to be numeric
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formating that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column. 

THESE ARE THE EXACT SAME INSTRUCTIONS FROM HW6! Please take this opportunity to reuse your code (if you got it right last time). 

In [2]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
    day, month, year = d.split('/')
    day, month, year = day.strip(), month.strip(), year.strip()
    if len(day) < 2: day = '0' + day
    if len(month) < 2: month = '0' + month
    if len(year) < 4: year = "20" + year
    standard_date = day + '.' + month + '.' + year
    return standard_date


In [3]:
df_melb['Date'] = df_melb['Date'].apply( standardize_date )
df_melb['unixtime'] = df_melb['Date'].apply(lambda x : int(calendar.timegm(time.strptime(x, "%d.%m.%Y"))))
df_melb = df_melb.drop(columns="Date")
df_melb.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,2,h,399000,8.7,3032,1,1.0,904,53.0,1985.0,1462579200
1,3,h,1241000,13.9,3165,1,1.0,643,NaN,NaN,1472342400
2,2,u,550000,3.0,3067,1,1.0,1521,NaN,NaN,1499472000
3,3,u,691000,8.4,3072,1,1.0,170,NaN,NaN,1498262400
4,2,u,657500,4.6,3122,1,1.0,728,73.0,1965.0,1479513600


## Q2 Calculating the prior probabilities
Calculate the prior probabilities for each possible "Type" in `df_melb` and populate a dictionary, `dict_priors`, where the key is the possible "Type" values and the value is the prior probabilities. Show the dictionary. Do not hardcode the possible values of "Type".  Don't forget about [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html). 

In [4]:
dict_priors = {}
types = df_melb['Type'].value_counts().to_frame()
for index, row in types.iterrows():
  dict_priors[index] = int(row['Type']) / len(df_melb)

dict_priors



{'h': 0.452, 't': 0.13, 'u': 0.418}

## Q3 Create a model for the distribution of all of the numeric attributes
For each class, and for each attribute calculate the sample mean and sample standard deviation.  You should store the model in a nested dictionary, `dict_nb_model`, such that `dict_nb_model['h']['Rooms']` is a tuple containing the mean and standard deviation for the target Type 'h' and the attribute 'Rooms'.  Show the model using the `display` function. You should ignore entries that are `NaN` in the mean and [standard deviation](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html) calculation. 

In [5]:
df2 = df_melb.copy()

df_h = df2[df2['Type'] == 'h']
df_h = df_h.drop(columns= ['Type'], axis=1)

df_u = df2[df2['Type'] == 'u']
df_u = df_u.drop(columns= ['Type'], axis=1)

df_t = df2[df2['Type'] == 't']
df_t = df_t.drop(columns= ['Type'], axis=1)

columns = df_melb.columns.values
columns = np.delete(columns, np.where(columns == 'Type'))

dfh = {}
dfu = {}
dft = {}

dataframes = [df_h, df_t, df_u]
dicts = [dfh, dft, dfu]

for i in range(0, 3):
  df_train = dataframes[i]
  
  for column in columns:
    average = df_train[column].mean(skipna=True)
    stdD = df_train[column].std(skipna=True)
    mstuple = (average,stdD)
    dicts[i][column] = mstuple


dict_nb_model = dict()
count = 0
for target in dict_priors.keys():
    dict_nb_model[target] = dicts[count]
    count += 1

In [6]:
display(dict_nb_model) 

{'h': {'Bathroom': (1.5619469026548674, 0.6720871086493075),
  'BuildingArea': (156.24339622641511, 54.62662837301434),
  'Car': (1.7777777777777777, 0.932759177140425),
  'Distance': (12.086725663716809, 7.397501132737295),
  'Landsize': (932.9646017699115, 3830.7934157687164),
  'Postcode': (3103.8982300884954, 98.35750345419703),
  'Price': (1189022.3451327435, 586296.5794417895),
  'Rooms': (3.269911504424779, 0.7258264201127756),
  'YearBuilt': (1954.900826446281, 32.461876347154686),
  'unixtime': (1485717578.761062, 13838562.050601462)},
 't': {'Bathroom': (1.1818181818181819, 0.42228151548662185),
  'BuildingArea': (83.85585585585585, 45.959438015166604),
  'Car': (1.1483253588516746, 0.47231993860296956),
  'Distance': (8.760287081339715, 5.609778714430755),
  'Landsize': (436.23444976076553, 1394.3403794653254),
  'Postcode': (3120.4545454545455, 87.18475679946482),
  'Price': (634207.1770334928, 217947.32866736987),
  'Rooms': (2.0430622009569377, 0.5908453859944267),
  'Yea

## Q4 Write a function that calculates the probability of a Gaussian
Given the mean ($\mu$), standard deviation ($\sigma$), and a observed point, `x`, return the probability.  
Use the formula $p(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ ([wiki](https://en.wikipedia.org/wiki/Normal_distribution)).  You should use [numpy's exp](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) function in your solution. 

In [7]:
def get_p( mu, sigma, x):
    first = 1 / (sigma * np.sqrt(2 * np.pi))
    second = np.exp(-0.5 * ((x - mu) / sigma)**2)
    return (first * second)

In [8]:
# Test it
p = get_p( 0, 2, 0.5)
p


0.19333405840142462

## Q5 Write the Naive Bayes classifier function
The Naive Bayes classifier function, `nb_class`, should take as a parameter the prior probability dictionary. `dict_priors`, the dictionary containing all of the gaussian distribution information for each attribue, `dict_nb_model`, and a single observation row (a series generated from iterrows) of the test dataframe. It should return a single target classification. For this problem, all of our attributes are numeric and modeled as Gaussians, so we don't worry about categorical data. Make sure to skip attributes that do not have a value in the observation.  Do not hardcode the possible classification types. 

In [9]:
def nb_class( dict_priors, dict_nb_model, observation):
    probs = {}
    for target in dict_priors.keys():
      probs[target] = dict_priors[target]
      x = dict_nb_model[target]
      for key in x.keys():
        p = get_p(x[key][0], x[key][1], observation[key])
        probs[target] *= p
    max_key = max(probs, key=probs.get)
    return (str(max_key))




## Q6 Calculate the accuracy using Naive Bayes classifier function on the test set
Load the test set from file, convert date to unix time and drop the date column, classify each row using your `nb_class`, and then show the accuracy on the test set. 

In [10]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')
df_test['Date'] = df_test['Date'].apply( standardize_date )
df_test['unixtime'] = df_test['Date'].apply(lambda x : int(calendar.timegm(time.strptime(x, "%d.%m.%Y"))))
df_test = df_test.drop(columns="Date")
df_test.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,3,h,1116000,17.9,3192,1,2.0,610,NaN,NaN,1498867200
1,3,h,2030000,11.2,3186,2,2.0,366,NaN,NaN,1472342400
2,3,h,1480000,10.7,3187,2,2.0,697,143.0,1925.0,1478476800
3,3,u,1203500,12.3,3166,2,2.0,311,127.0,2000.0,1495843200
4,3,h,540000,14.7,3030,2,2.0,353,135.0,2011.0,1504396800


In [11]:
predictions = []
correct = 0
for (indx,row) in df_test.iterrows():
    predictions.append(nb_class(dict_priors, dict_nb_model, row))
    if row['Type'] == nb_class(dict_priors, dict_nb_model, row):
      correct += 1
    

In [12]:
acc = correct / len(df_test)
acc

0.63

In [13]:
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 63.00%


## Use scikit-learn to do the same thing!

Now we understand the inner workings of the Naive Bayes algorithm, let's compare our results to [scikit-learn's Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) implementation. Use the [GaussianNB](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) to train using the `df_melb`dataframe and test using the `df_test` dataframe. Remember to split `df_melb` into a `df_X` with the numerical attributes, and a `s_y` with the target column. On the `df_melb` frame you will have to fill the empty attributes via imputation since the scikit-learn library can not handle missing values.  Use the same method you used in the last homework (filling the training data with the mean of the non-nan values). 

Answer the following in a markdown cell: do you think imputation hurt or helped the classifier?

In [14]:
# Imputation training
dict_imputation = dict()
# your code here
df3 = df_melb.copy()
for column in df_train.columns:
  if column != "Type":
    df3[column].fillna(df3[column].mean(), inplace = True)
    dict_imputation[column] = df3[column].mean()


# Imputation - apply on the test data
# your code here
for column in df_test.columns:
  if column != "Type":
    df_test[column].fillna(dict_imputation[column], inplace = True)


# Seperate the attributes from the target_col
# your code here
s_y, df_X = df3['Type'], df3.drop(columns = 'Type')
df_test2 = df_test.copy()
df_test = df_test.drop(columns = 'Type')


In [15]:
gnb = GaussianNB()
# your code here
prediction2 = gnb.fit(df_X, s_y).predict(df_test)
prediction2

array(['t', 'h', 't', 't', 'u', 'u', 't', 'u', 't', 'u', 'u', 't', 'u',
       'u', 'u', 't', 'h', 'h', 't', 'u', 't', 't', 'u', 'h', 't', 't',
       'u', 'h', 'h', 't', 't', 't', 't', 'u', 'h', 'u', 'u', 'h', 't',
       't', 't', 'u', 'u', 'u', 'h', 'h', 'u', 'u', 't', 'u', 't', 't',
       't', 'u', 'u', 'u', 't', 't', 'u', 'u', 't', 'u', 'u', 'u', 't',
       'h', 't', 'u', 'u', 'u', 'u', 't', 'u', 't', 'u', 't', 'u', 'u',
       'u', 't', 'u', 'h', 'u', 'u', 'u', 'u', 't', 't', 'u', 'u', 'u',
       'u', 'u', 'h', 'u', 'u', 'u', 't', 'h', 'u'], dtype='<U1')

In [16]:
actual = df_test2['Type'].tolist()
correct = 0
for i in range(0,100):

  if actual[i] == prediction2[i]:
    correct += 1
acc = correct / len(df_test2)

# acc = # your code here
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 37.00%


## ANSWER TO "do you think imputation hurt or helped the classifier?" 

we think imputation hurt the classifier since the accuracy dropped. 